In [14]:
import requests
import pandas as pd
import numpy as np
import folium
from folium.plugins import MiniMap
import requests
import folium
import collections
 

##카카오 API
def whole_region(keyword, start_x,start_y,end_x,end_y):
    #print(start_x,start_y,end_x,end_y)
    page_num = 1
    # 데이터가 담길 리스트
    all_data_list = []
    
    while(1):
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
        params = {'query': keyword,'page': page_num, 
                 'rect': f'{start_x},{start_y},{end_x},{end_y}'}
        headers = {"Authorization": "KakaoAK fb2508f480c59758b32b98a82429becc"}
        ## 입력예시 -->> headers = {"Authorization": "KakaoAK fb2508f480c59758b32b98a82429becc"}
        resp = requests.get(url, params=params, headers=headers)

        search_count = resp.json()['meta']['total_count']
        print('총 개수',search_count)
        
        if search_count > 45:
            print('좌표 4등분')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            ## 4등분 중 왼쪽 아래
            all_data_list.extend(whole_region(keyword, start_x,start_y,dividing_x,dividing_y))
            ## 4등분 중 오른쪽 아래
            all_data_list.extend(whole_region(keyword, dividing_x,start_y,end_x,dividing_y))
            ## 4등분 중 왼쪽 위
            all_data_list.extend(whole_region(keyword, start_x,dividing_y,dividing_x,end_y))
            ## 4등분 중 오른쪽 위
            all_data_list.extend(whole_region(keyword, dividing_x,dividing_y,end_x,end_y))
            return all_data_list
        
        else:
            if resp.json()['meta']['is_end']:
                all_data_list.extend(resp.json()['documents'])
                return all_data_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('다음페이지')
                page_num += 1
                all_data_list.extend(resp.json()['documents'])
 

def overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y):
    # 최종 데이터가 담길 리스트
    overlapped_result = []

    # 지도를 사각형으로 나누면서 데이터 받아옴
    for i in range(1,num_x+1):   ## 1,10
        end_x = start_x + next_x
        initial_start_y = start_y
        for j in range(1,num_y+1):  ## 1,6
            end_y = initial_start_y + next_y
            each_result= whole_region(keyword, start_x,initial_start_y,end_x,end_y)
            overlapped_result.extend(each_result)
            initial_start_y = end_y
        start_x = end_x
    
    return overlapped_result
 

def make_map(dfs):
    # 지도 생성하기
    m = folium.Map(location=[33.4935,126.6266],   # 기준좌표: 제주어딘가
                   zoom_start=12)

    # 미니맵 추가
    minimap = MiniMap() 
    m.add_child(minimap)

    # 마커
    for i in range(len(dfs)):
        folium.Marker([df['Y'][i],df['X'][i]],    # 제주 어딘가
                  tooltip=dfs['stores'][i],
                  popup=dfs['place_url'][i],
                  ).add_to(m)
    return m

In [15]:
# 시작 x 좌표 및 증가값
keyword = '식당'

start_x = 126.12484043304154
start_y = 33.18739686999

next_x = 0.02
next_y = 0.02
num_x = 180
num_y = 90
 

overlapped_result = overlapped_data(keyword, start_x, start_y, next_x, next_y, num_x, num_y)

# 최종 데이터가 담긴 리스트 중복값 제거
results = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in overlapped_result)))
X = []
Y = []
stores = []
road_address = []
place_url = []
ID = []
for place in results:
    
    X.append(float(place['x']))
    Y.append(float(place['y']))
    stores.append(place['place_name'])
    road_address.append(place['road_address_name'])
    place_url.append(place['place_url'])
    ID.append(place['id'])

    ar = np.array([ID,stores, X, Y, road_address,place_url]).T
    df = pd.DataFrame(ar, columns = ['ID','stores', 'X', 'Y','road_address','place_url'])
    
print('total_reuslt_number = ',len(df))
 

display(df.head())
 



KeyError: 'meta'

In [80]:
make_map(df)

In [13]:
df.tail(50)

,ID,stores,X,Y,road_address,place_url
3968,772919858,지세포방파제주차장,128.717785951071,34.8372702897372,경남 거제시 일운면 지세포해안로 242,http://place.map.kakao.com/772919858
3969,27387391,지세포항해안도로주차장,128.71673977657,34.8355293963972,,http://place.map.kakao.com/27387391
3970,26494530,장승포여객터미널 주차장,128.723525905842,34.8636393439332,경남 거제시 장승로 98,http://place.map.kakao.com/26494530
3971,449645811,옥림바다 주차장,128.715128693463,34.8491269895188,,http://place.map.kakao.com/449645811
3972,20593046,주차장,128.722262182222,34.8546338175873,경남 거제시 마전1길 29,http://place.map.kakao.com/20593046
3973,18118843,거제문화예술회관 주차장1,128.723684700312,34.8678946492616,,http://place.map.kakao.com/18118843
3974,27349579,대우병원 주차장3,128.722466742058,34.8717837284707,경남 거제시 두모길 16,http://place.map.kakao.com/27349579
3975,27349577,대우병원 주차장1,128.721254447238,34.8719631275169,경남 거제시 두모길 16,http://place.map.kakao.com/27349577
3976,18118842,거제문화예술회관 주차장2,128.722959064913,34.8677066587287,경남 거제시 장승로 145,http://place.map.kakao.com/18118842
3977,20568700,주차장,128.71983166282,34.8727475349333,,http://place.map.kakao.com/20568700


In [14]:
df.to_csv('주차장.csv')

In [87]:
df.drop_duplicate(['ID'])

AttributeError: 'DataFrame' object has no attribute 'drop_duplicate'

In [95]:
df2 = df.drop(df.index[17359:])

In [97]:
df2.to_csv('제주식당.csv')

In [19]:
resp.json()

NameError: name 'resp' is not defined